In [4]:
!pip install xgboost

Keyring is skipped due to an exception: 'keyring.backends'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 3.4 MB/s eta 0:00:0000:0100:01


In [5]:
import pandas as pd
from xgboost import XGBClassifier

In [7]:
train_df = pd.read_csv('s3://sagemaker-us-east-1-602900100639/data/kkbox-customer-churn-model/processed/train/part-00000-bbef07c9-1304-47fa-8f34-2c2fbb5257fe-c000.csv', 
                       names=['msno', 'is_churn', 'regist_trans', 'mst_frq_plan_days', 'revenue', 'regist_cancels', 'bd', 'tenure', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs', 'city', 'gender', 'registered_via', 'qtr_trans', 'mst_frq_pay_met', 'is_auto_renew'])
test_df = pd.read_csv('s3://sagemaker-us-east-1-602900100639/data/kkbox-customer-churn-model/processed/test/part-00000-064760ee-4efc-4e0b-8122-e787e54232bd-c000.csv',
                      names=['msno', 'is_churn', 'regist_trans', 'mst_frq_plan_days', 'revenue', 'regist_cancels', 'bd', 'tenure', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs', 'city', 'gender', 'registered_via', 'qtr_trans', 'mst_frq_pay_met', 'is_auto_renew'])

In [17]:
model = XGBClassifier()

In [18]:
x = train_df.drop(["msno"],axis=1).iloc[:, 1:]
y = train_df.loc[:, "is_churn"]

In [19]:
model.fit(x,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import numpy as np

In [25]:
x_test = test_df.drop(["msno"],axis=1).iloc[:, 1:]
y_test = test_df.loc[:, "is_churn"]

In [26]:
y_pred = model.predict(x_test)
y_pred_proba=model.predict_proba(x_test)[:,1]

In [27]:
#Printing Metrics

print('\nConfusion Matrix')
print(confusion_matrix(y_test, y_pred))


print('\nScores')
print('------------------------')
print('AUC:',np.round(roc_auc_score(y_test,y_pred_proba)*100,2),'%')
print('Accuracy:',np.round(accuracy_score(y_test,y_pred)*100,2),'%')
print('Precision:',np.round(precision_score(y_test,y_pred)*100,2),'%')
print('Recall:',np.round(recall_score(y_test,y_pred)*100,2),'%')
print('F1 score:',np.round(f1_score(y_test,y_pred)*100,2))


Confusion Matrix
[[158833  26995]
 [   608  12053]]

Scores
------------------------
AUC: 96.12 %
Accuracy: 86.09 %
Precision: 30.87 %
Recall: 95.2 %
F1 score: 46.62
